In [1]:
import duckdb as db

In [2]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [3]:
urls = {
    'name_basics' : 'https://datasets.imdbws.com/name.basics.tsv.gz',
    'title_basics' : 'https://datasets.imdbws.com/title.basics.tsv.gz',
    'title_crew' : 'https://datasets.imdbws.com/title.crew.tsv.gz',
    'title_ratings' : 'https://datasets.imdbws.com/title.ratings.tsv.gz',
    'title_akas' : 'https://datasets.imdbws.com/title.akas.tsv.gz',
    'tmdb' : '../data/tmdb_full.csv'
}

for name, url in urls.items():
    if name == 'title_basics':
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
                where titletype = 'movie'
            """
        )
    else:
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
            """
        )


In [4]:
sql(
    """
    show tables

"""
)

,name
0,name_basics
1,title_akas
2,title_basics
3,title_crew
4,title_ratings
5,tmdb


In [5]:
sql(
    """
    select * from title_basics
    limit 1
    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance


In [24]:
sql(
    """ 
    select primaryTitle as Title,
    ta.Region as Region, genres,
    startYear as Year,
    runtimeMinutes as Runtime,
    nb.primaryName as Director
    from title_basics tb
    join title_akas ta on tconst = titleId
    join title_crew using (tconst)
    join name_basics nb on directors = nconst
    where region like 'US'
"""
)

,Title,Region,genres,Year,Runtime,Director
0,27: The Cursed Club,US,"Action,Comedy,Crime",2018,80,Nicanor Loreti
1,The Birch-Tree Meadow,US,Drama,2003,91,Marceline Loridan Ivens
2,My Brother's Gun,US,"Crime,Drama,Romance",1997,93,Ray Loriga
3,The Age of Love,US,"Documentary,Romance",2014,79,Steven Loring
4,He Hired the Boss,US,"Action,Comedy",1943,72,Thomas Z. Loring
...,...,...,...,...,...,...
307427,Keepers,US,"Sci-Fi,Thriller",\N,\N,Raul Abarca
307428,Dream Thief,US,"Drama,Fantasy,Mystery",2024,\N,Raul Abarca
307429,A Christmas Jingle,US,Fantasy,2025,\N,Raul Abarca
307430,Dark Cupid,US,"Drama,Fantasy,Mystery",\N,\N,Raul Abarca
